# **13.5 Raising_Exceptions**

So far we've been *catching* exceptions that Python raises for us. But sometimes you need to raise one yourself — when a player passes an invalid team size, a Pokemon's level is out of range, or a save file is corrupted. Using `raise` lets you enforce rules and communicate problems clearly to whoever calls your code. In this lesson you'll learn how and when to raise exceptions intentionally in your Pokemon programs.

---

## **The raise Statement**

`raise ExceptionType("message")` immediately stops the current function and sends an exception up the call stack. The caller can catch it or let it propagate further. The message string you provide is what shows up in the traceback — make it specific and helpful.

In [ ]:
def set_pokemon_level(level: int) -> int:
    """
    Validate and return a Pokemon level.
    Raises ValueError with a clear message if the level is invalid.
    """
    if not isinstance(level, int):
        raise TypeError(f"Level must be an int, got {type(level).__name__}")
    if level < 1:
        raise ValueError(f"Level must be at least 1, got {level}")
    if level > 100:
        raise ValueError(f"Level cannot exceed 100, got {level}")
    return level

# Valid input
print(set_pokemon_level(25))

# Invalid inputs — each raises a different, informative exception
for bad in [0, 101, "fifty", 3.5]:
    try:
        set_pokemon_level(bad)
    except (ValueError, TypeError) as e:
        print(f"  ✗ {type(e).__name__}: {e}")

---

## **Choosing the Right Exception Type**

Python's built-in exception types each have a specific meaning. Reusing the right one helps callers know exactly what went wrong — and lets them use `except ValueError` precisely instead of catching everything.

In [ ]:
# Guide to choosing the right built-in exception

def add_to_team(team: list, pokemon: str) -> list:
    """Add a Pokemon to the team — raises TypeError or ValueError for bad input."""
    # Wrong TYPE of argument
    if not isinstance(pokemon, str):
        raise TypeError(f"pokemon must be a str, got {type(pokemon).__name__}")
    # Logically invalid VALUE
    if len(team) >= 6:
        raise ValueError("Team is full — cannot have more than 6 Pokemon")
    team.append(pokemon)
    return team

def lookup_pokemon(pokedex: dict, name: str) -> dict:
    """Lookup a Pokemon — raises KeyError for unknown names."""
    if name not in pokedex:
        raise KeyError(f"'{name}' is not registered in the Pokedex")
    return pokedex[name]

def calculate_damage(power: int, level: int) -> int:
    """Calculate damage — raises ZeroDivisionError if level is 0."""
    if level == 0:
        raise ZeroDivisionError("Pokemon level cannot be 0 in damage formula")
    return power * level // 10

# Test each
team = []
try:
    add_to_team(team, 123)          # TypeError
except TypeError as e:
    print(f"TypeError: {e}")

try:
    lookup_pokemon({}, "Mewtwo")    # KeyError
except KeyError as e:
    print(f"KeyError: {e}")

---

## **raise Without Arguments — Re-raising**

A bare `raise` inside an `except` block re-raises the current exception unchanged. This lets you inspect or log an error, then pass it on to the caller to deal with.

In [ ]:
import datetime

battle_log = []

def log_and_reraise(operation: str, func, *args):
    """Log any exception that occurs, then re-raise it unchanged."""
    try:
        return func(*args)
    except Exception as e:
        battle_log.append({
            'time': str(datetime.datetime.now()),
            'op': operation,
            'error': f"{type(e).__name__}: {e}"
        })
        raise   # ← bare raise: same exception continues upward

# Outer code catches the re-raised exception
try:
    log_and_reraise("parse_level", int, "Pikachu")
except ValueError:
    print("Outer handler: recovered from ValueError")

print(f"\nLog: {battle_log}")

---

## **raise ... from — Exception Chaining**

When you catch one exception and raise a different one, use `raise NewException(...) from original_exception` to preserve the original context. This tells Python (and future readers) that the new error was *caused by* the original.

In [ ]:
class PokedexError(Exception):
    """Raised when a Pokedex operation fails."""

def load_pokedex(filename: str) -> dict:
    """
    Load Pokedex data from a JSON file.
    Wraps low-level FileNotFoundError in a more meaningful PokedexError.
    """
    import json
    try:
        with open(filename) as f:
            return json.load(f)
    except FileNotFoundError as e:
        # 'from e' chains the original cause — visible in the traceback
        raise PokedexError(
            f"Cannot load Pokedex: '{filename}' does not exist"
        ) from e

try:
    data = load_pokedex("kanto.json")
except PokedexError as e:
    print(f"PokedexError caught: {e}")
    # e.__cause__ holds the original FileNotFoundError
    print(f"Caused by: {e.__cause__}")

---

## **Assertions — raise for Programmer Mistakes**

The `assert` statement is a shorthand for raising `AssertionError` when a condition is False. Use it to check **internal** assumptions your code makes — things that should *never* be false if the code is correct. Don't use it for validating user input.

In [ ]:
def calculate_experience(level: int, base_exp: int) -> int:
    """
    Calculate experience for a given level.
    Assertions guard internal logic assumptions.
    """
    # assert condition, "message if condition is False"
    assert 1 <= level <= 100, f"Internal error: level {level} out of range"
    assert base_exp > 0, f"Internal error: base_exp must be positive, got {base_exp}"

    return base_exp * (level ** 3) // 100

# Valid call
print(f"Pikachu exp at level 25: {calculate_experience(25, 112)}")

# Assertion failure — programmer passed bad internal data
try:
    calculate_experience(0, 100)  # level 0 violates assertion
except AssertionError as e:
    print(f"AssertionError: {e}")

# Note: assertions can be disabled with 'python -O script.py'
# So never rely on them for user-facing validation

---

## **Practical Example: Full Validation System**

In [ ]:
VALID_TYPES = {"Normal", "Fire", "Water", "Grass", "Electric",
               "Ice", "Fighting", "Poison", "Ground", "Flying",
               "Psychic", "Bug", "Rock", "Ghost", "Dragon"}

def create_pokemon(name: str, ptype: str, level: int, hp: int) -> dict:
    """
    Create a validated Pokemon dictionary.
    Raises TypeError or ValueError with clear messages for any invalid argument.
    """
    # Type checks
    if not isinstance(name, str):
        raise TypeError(f"name must be str, got {type(name).__name__}")
    if not isinstance(ptype, str):
        raise TypeError(f"type must be str, got {type(ptype).__name__}")
    if not isinstance(level, int):
        raise TypeError(f"level must be int, got {type(level).__name__}")
    if not isinstance(hp, int):
        raise TypeError(f"hp must be int, got {type(hp).__name__}")

    # Value checks
    if not name.strip():
        raise ValueError("name cannot be empty")
    if ptype not in VALID_TYPES:
        raise ValueError(f"'{ptype}' is not a valid Pokemon type")
    if not (1 <= level <= 100):
        raise ValueError(f"level must be 1–100, got {level}")
    if hp <= 0:
        raise ValueError(f"hp must be positive, got {hp}")

    return {"name": name, "type": ptype, "level": level, "hp": hp}

test_cases = [
    ("Pikachu",  "Electric", 25, 35),    # Valid
    ("Charizard","Flame",    36, 78),    # Invalid type
    ("Blastoise","Water",   150, 79),    # Level out of range
    (123,         "Fire",    10, 40),    # Name not a string
]

for args in test_cases:
    try:
        p = create_pokemon(*args)
        print(f"  ✓ Created: {p['name']} Lv.{p['level']}")
    except (TypeError, ValueError) as e:
        print(f"  ✗ {type(e).__name__}: {e}")

---

## **Practice Exercises**

### **Task 1: Raise ValueError**

Raise `ValueError` if HP is not between 1 and 500.

**Expected Output:**
```
ValueError: HP must be 1–500, got 0
```

In [ ]:
def set_hp(hp):
    # Your code here:
    pass

try:
    set_hp(0)
except ValueError as e:
    print(f"ValueError: {e}")

### **Task 2: Raise TypeError**

Raise `TypeError` if the pokemon name is not a string.

**Expected Output:**
```
TypeError: name must be str, got int
```

In [ ]:
def set_name(name):
    # Your code here:
    pass

try:
    set_name(25)
except TypeError as e:
    print(f"TypeError: {e}")

### **Task 3: Guard Team Size**

Raise `ValueError` when adding a 7th Pokemon to the team.

**Expected Output:**
```
Team is full!
```

In [ ]:
team = ["Pikachu", "Charizard", "Blastoise", "Venusaur", "Gengar", "Snorlax"]

# Your code here:


### **Task 4: Bare Raise Re-raise**

Log the error, then re-raise it with a bare `raise`.

**Expected Output:**
```
[LOG] ValueError caught
Outer: invalid level
```

In [ ]:
# Your code here:


### **Task 5: Validate Pokemon Type**

Raise `ValueError` if the type is not in the valid list.

**Expected Output:**
```
ValueError: 'Flame' is not a valid type
```

In [ ]:
VALID_TYPES = ["Fire", "Water", "Grass", "Electric", "Normal"]

# Your code here:


### **Task 6: assert Statement**

Use `assert` to verify an internal assumption — that the damage value is non-negative.

**Expected Output:**
```
AssertionError: damage must be non-negative
```

In [ ]:
def apply_damage(pokemon, damage):
    # Your assert here:
    pokemon['hp'] -= damage
    return pokemon

try:
    apply_damage({"name": "Pikachu", "hp": 35}, -10)
except AssertionError as e:
    print(f"AssertionError: {e}")

### **Task 7: raise ... from**

Catch a `KeyError` and re-raise it as a `ValueError` with chaining using `from`.

**Expected Output:**
```
ValueError: Pikachu not found in Pokedex
```

In [ ]:
pokedex = {"Charizard": "Fire"}

# Your code here:


### **Task 8: Multiple Validations**

Validate name, level, and type in one function, raising appropriate errors.

**Expected Output:**
```
✓ Pikachu Electric 25
✗ ValueError: level out of range
✗ TypeError: name must be str
```

In [ ]:
# Your code here:


### **Task 9: raise in Loop**

Process a list of levels, raising `ValueError` and catching it to skip bad entries.

**Expected Output:**
```
Valid: [25, 36, 8]
Skipped: ['bad', 150]
```

In [ ]:
inputs = ["25", "bad", "36", "150", "8"]

# Your code here:


### **Task 10: Descriptive Messages**

Write a validator that provides a different, helpful message for each validation failure.

**Expected Output:**
```
✗ name '' is empty
✗ level 0 below minimum of 1
✗ hp -5 must be positive
```

In [ ]:
bad_inputs = [
    ("",        "Electric", 25,  35),   # empty name
    ("Pikachu", "Electric",  0,  35),   # level too low
    ("Pikachu", "Electric", 25,  -5),   # negative hp
]

# Your code here:


---

## **Summary**

- `raise ExceptionType("message")` raises an exception immediately
- Choose the most specific built-in type that fits the problem
- `TypeError` — wrong argument type
- `ValueError` — right type but logically invalid value
- `KeyError` — missing key in dict-like lookups
- Bare `raise` re-raises the current exception (for logging + rethrow)
- `raise NewError(...) from original` chains exceptions, preserving context
- `assert condition, "msg"` — for internal programmer assumptions only
- Write clear, specific messages — they appear directly in tracebacks

---

## **Quick Reference**

```python
# Raise with message
raise ValueError("level must be 1–100")
raise TypeError(f"expected str, got {type(x).__name__}")

# Re-raise current exception
except Exception:
    log_error()
    raise

# Chain exceptions
except FileNotFoundError as e:
    raise PokedexError("file missing") from e

# Assert internal assumption
assert value > 0, f"expected positive, got {value}"

# Guard pattern
def validate(x):
    if not isinstance(x, int):
        raise TypeError(f"expected int, got {type(x).__name__}")
    if x < 0:
        raise ValueError(f"must be non-negative, got {x}")
    return x
```